In [1]:
import sys
import numpy as np

from keras.datasets import mnist

In [16]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the training data
train_images = x_train[0:1000].reshape(1000, 28*28) / 255
train_labels = np.zeros((len(y_train[0:1000]), 10))
for i, l in enumerate(y_train[0:1000]):
    train_labels[i][l] = 1

# Assign labels to one_hot_labels
labels = train_labels

# Preprocess the test data
test_images = x_test.reshape(len(x_test), 28*28) / 255
test_labels = np.zeros((len(y_test), 10))
for i, l in enumerate(y_test):
    test_labels[i][l] = 1

np.random.seed(1)
relu = lambda x: (x >= 0) * x
relu2deriv = lambda x: x >= 0

alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.005, 350, 40, 784, 10)
weights_0_1 = 0.2*np.random.random((pixels_per_image, hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1

train_acc_list = {}
test_acc_list = {}

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    for i in range(len(train_images)):
        layer_0 = train_images[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)

        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))

        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)*relu2deriv(layer_1)

        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    sys.stdout.write("\r" + \
                     " I:" + str(j) + \
                     " Error:" + str(error/float(len(train_images)))[0:5] + \
                     " Correct:" + str(correct_cnt/float(len(train_images))))
    if (j % 10 == 0 or j == iterations-1):
        train_acc_list[j] = correct_cnt/float(len(train_images))
        error, correct_cnt = (0.0, 0)
        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0, weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)

            error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
        sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] + \
                         " Test-Acc:" + str(correct_cnt/float(len(test_images))) + "\n")
        test_acc_list[j] = correct_cnt/float(len(test_images))


        

 I:0 Error:0.722 Correct:0.537 Test-Err:0.601 Test-Acc:0.6488
 I:10 Error:0.312 Correct:0.901 Test-Err:0.420 Test-Acc:0.8114
 I:20 Error:0.260 Correct:0.937 Test-Err:0.414 Test-Acc:0.8111
 I:30 Error:0.232 Correct:0.946 Test-Err:0.417 Test-Acc:0.8066
 I:40 Error:0.215 Correct:0.956 Test-Err:0.426 Test-Acc:0.8019
 I:50 Error:0.204 Correct:0.966 Test-Err:0.437 Test-Acc:0.7982
 I:60 Error:0.194 Correct:0.967 Test-Err:0.448 Test-Acc:0.7921
 I:70 Error:0.186 Correct:0.975 Test-Err:0.458 Test-Acc:0.7864
 I:80 Error:0.179 Correct:0.979 Test-Err:0.466 Test-Acc:0.7817
 I:90 Error:0.172 Correct:0.981 Test-Err:0.474 Test-Acc:0.7758
 I:100 Error:0.166 Correct:0.984 Test-Err:0.482 Test-Acc:0.7706
 I:110 Error:0.161 Correct:0.984 Test-Err:0.489 Test-Acc:0.7686
 I:120 Error:0.157 Correct:0.986 Test-Err:0.496 Test-Acc:0.766
 I:130 Error:0.153 Correct:0.999 Test-Err:0.502 Test-Acc:0.7622
 I:140 Error:0.149 Correct:0.991 Test-Err:0.508 Test-Acc:0.758
 I:150 Error:0.145 Correct:0.991 Test-Err:0.513 Test-

In [17]:
correct_cnt/float(len(train_images))

7.073

In [18]:
# draw the graph in the end of the training and testing accuracy in plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(train_acc_list.keys()), y=list(train_acc_list.values()), mode='lines+markers', name='Train Accuracy'))
fig.add_trace(go.Scatter(x=list(test_acc_list.keys()), y=list(test_acc_list.values()), mode='lines+markers', name='Test Accuracy'))
fig.update_layout(title='Train and Test Accuracy', xaxis_title='Iterations', yaxis_title='Accuracy')
fig.show()
